### Init Ray


In [ ]:
import ray
#from ray.rllib.env.env_context import EnvContext
#from ray.rllib.algorithms import appo
#from ray.rllib.algorithms.appo import APPOConfig
if not ray.is_initialized():
    ray.init()
    assert ray.is_initialized()
    

In [ ]:
from ray.rllib.algorithms.appo import APPOConfig
config = (
    APPOConfig()
    .rollouts(horizon=10000)
    .environment(
        MyEnv,
        env_config={}
    )
)

# ray.rllib

In [ ]:
import gym, ray
from ray.rllib.algorithms import ppo
#import ray.rllib.agents.ppo as ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print

In [ ]:
#dir(ray)
ray.shutdown()
ray.init(ignore_reinit_error=True)

In [ ]:
config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

In [ ]:
algo = ppo.PPO(env=MyEnv, config=config) 

In [ ]:
#print("Dashboard URL: http://{}".format(ray.get_webui_url()))

In [ ]:
#config = PPOConfig()#.rollouts(horizon=200) 
config={
        "env": MyEnv,
        "num_workers": 30,
        "num_cpus_per_worker": 0.5,
        "env_config":{
            "max_steps": 1000,
            "export_frames": False,
            "export_states": False,
            # "reward_mode": "continuous",
            # "env_flipped": True,
            # "env_flipmode": True,
            }
        }

In [ ]:
#algo = ppo.PPO(env=MyEnv, config={"env_config": {},  # config to pass to env class
#})

In [ ]:
mean_ppo = []
for _ in range(25):
    result = algo.train()
    print("episode reward mean:", _, result['episode_reward_mean'])
    mean_ppo.append(result['episode_reward_mean'])

In [ ]:
from ray import tune
tune.run(
    "SAC", # reinforced learning agent
    name = "Training2",
    checkpoint_freq = 100,
    checkpoint_at_end = True,
    local_dir = r'./ray_results/',
    config={
        "env": MyEnv,
        "num_workers": 30,
        "num_cpus_per_worker": 0.5,
        "env_config":{
            "max_steps": 1000,
            "export_frames": False,
            "export_states": False,
            # "reward_mode": "continuous",
            # "env_flipped": True,
            # "env_flipmode": True,
            }
        },
    stop = {
        "timesteps_total": 5_000_000,
        },
    )

In [ ]:
N_ITER = 30
s = "{:3d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:6.2f} saved {}"

for n in range(N_ITER):
  result = algo.train()
  #file_name = agent.save(CHECKPOINT_ROOT)

  print(s.format(
    n + 1,
    result["episode_reward_min"],
    result["episode_reward_mean"],
    result["episode_reward_max"],
    result["episode_len_mean"],
    file_name
   ))